In [1]:
import sagemaker
import boto3

# Get the default S3 bucket created by SageMaker
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/fine-tuning-data' # A folder in your S3 bucket

# Upload the local CSV file to S3
input_s3_path = sagemaker_session.upload_data(path='Reviews.csv', bucket=bucket, key_prefix=prefix)

print(f"Data uploaded to: {input_s3_path}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Data uploaded to: s3://sagemaker-us-east-1-059006397895/sagemaker/fine-tuning-data/Reviews.csv


In [2]:
from sagemaker.pytorch import PyTorch

# Get the IAM role for the notebook
role = sagemaker.get_execution_role()

# Create a PyTorch Estimator
estimator = PyTorch(
    entry_point='train.py',            # Your training script
    source_dir='./source',             # The directory of your script
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',       # A powerful instance for training
    framework_version='1.13.1',
    py_version='py39',
    requirements_file='./source/requirements.txt',
    hyperparameters={'epochs': 1}
)

# Launch the training job
estimator.fit({'train': input_s3_path})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-08-26-20-29-32-740


2025-08-26 20:29:37 Starting - Starting the training job...
2025-08-26 20:29:51 Starting - Preparing the instances for training...
2025-08-26 20:30:17 Downloading - Downloading input data......
2025-08-26 20:31:13 Downloading - Downloading the training image......
2025-08-26 20:32:24 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.9/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.9/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
2025-08-26

In [3]:
# Get the S3 path of the trained model artifact (model.tar.gz)
model_s3_path = estimator.model_data
print(f"Trained model is located at: {model_s3_path}")

Trained model is located at: s3://sagemaker-us-east-1-059006397895/pytorch-training-2025-08-26-20-29-32-740/output/model.tar.gz


In [8]:
from sagemaker.pytorch.model import PyTorchModel

# Create a SageMaker Model object from the trained artifact
model = PyTorchModel(
    model_data=model_s3_path,      # Path to your trained model in S3
    role=role,                     # The same IAM role
    entry_point='inference.py',    # Your inference script
    source_dir='./source_deploy',  # The directory of the script
    framework_version='1.13.1',
    py_version='py39'
)

# Deploy the model to a real-time endpoint
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium' # A small instance is fine for hosting
)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-059006397895/pytorch-training-2025-08-26-20-29-32-740/output/model.tar.gz), script artifact (./source_deploy), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-059006397895/pytorch-inference-2025-08-26-21-01-19-712/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2025-08-26-21-01-41-208
INFO:sagemaker:Creating endpoint-config with name pytorch-inference-2025-08-26-21-01-41-724
INFO:sagemaker:Creating endpoint with name pytorch-inference-2025-08-26-21-01-41-724


-----------------------------------------------------------------------------!

In [11]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# The text you want to analyze
test_data = {"text": "This movie was absolutely fantastic, the best I have seen all year!"}

# Tell the predictor to send data as JSON
predictor.serializer = JSONSerializer()

# Tell the predictor to expect a JSON response back
predictor.deserializer = JSONDeserializer()

# Get a prediction
response = predictor.predict(test_data)
print(response)

[{'label': 'LABEL_1', 'score': 0.7977247834205627}]


In [18]:
import boto3
import json
import time
import zipfile
import os

# --- 1. Configuration ---
# Get the current execution role to reuse for Lambda
try:
    role_arn = sagemaker.get_execution_role()
except NameError: # If sagemaker library is not imported, get role differently
    iam_client = boto3.client('iam')
    role_arn = iam_client.get_role(RoleName='AmazonSageMaker-ExecutionRole-YYYYMMDDTHHMMSS')['Role']['Arn'] # Replace with your actual role name if different

# Get the SageMaker endpoint name from the previously deployed predictor
# predictor is the variable from the last step where you ran model.deploy()
sagemaker_endpoint_name = predictor.endpoint_name 

lambda_function_name = "sagemaker-proxy-lambda"
api_name = "sagemaker-proxy-api"
region = boto3.Session().region_name

# --- 2. Create the Lambda Function Code ---
lambda_code = f"""
import os
import boto3
import json

ENDPOINT_NAME = '{sagemaker_endpoint_name}'
sagemaker_runtime = boto3.client('sagemaker-runtime')

def lambda_handler(event, context):
    try:
        body = json.loads(event.get("body", "{{}}"))
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            ContentType='application/json',
            Body=json.dumps(body)
        )
        result = response['Body'].read().decode('utf-8')
        return {{
            'statusCode': 200,
            'headers': {{'Content-Type': 'application/json'}},
            'body': result
        }}
    except Exception as e:
        print(e)
        return {{'statusCode': 500, 'body': json.dumps('Error invoking endpoint.')}}
"""

# Write the code to a file
with open("lambda_handler.py", "w") as f:
    f.write(lambda_code)

# --- 3. Create the Deployment Package (Zip File) ---
with zipfile.ZipFile('deployment_package.zip', 'w') as z:
    z.write('lambda_handler.py')

# --- 4. Attach the necessary SageMaker policy to the IAM role ---
iam_client = boto3.client('iam')
role_name = role_arn.split('/')[-1]
sagemaker_invoke_policy_arn = "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess" # For simplicity; a more restrictive policy is better for production
iam_client.attach_role_policy(RoleName=role_name, PolicyArn=sagemaker_invoke_policy_arn)
print(f"Attached SageMaker policy to role: {role_name}")
# It can take a few moments for the policy to attach and propagate
print("Waiting for IAM policy propagation...")
time.sleep(15) 

# --- 5. Create the Lambda Function ---
lambda_client = boto3.client('lambda')

with open('deployment_package.zip', 'rb') as f:
    zipped_code = f.read()

try:
    # Delete the function if it already exists
    lambda_client.delete_function(FunctionName=lambda_function_name)
    print("Deleted existing Lambda function. Recreating...")
    time.sleep(5)
except lambda_client.exceptions.ResourceNotFoundException:
    pass # Function doesn't exist, which is fine

lambda_response = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.9',
    Role=role_arn,
    Handler='lambda_handler.lambda_handler',
    Code={'ZipFile': zipped_code},
    Timeout=60
)
lambda_arn = lambda_response['FunctionArn']
print(f"Lambda function created with ARN: {lambda_arn}")

# --- 6. Create the API Gateway (HTTP API) ---
apigw_client = boto3.client('apigatewayv2')

try:
    # Find and delete if it already exists
    apis = apigw_client.get_apis()['Items']
    for api in apis:
        if api['Name'] == api_name:
            apigw_client.delete_api(ApiId=api['ApiId'])
            print("Deleted existing API Gateway. Recreating...")
            time.sleep(5)
except Exception as e:
    pass

api_response = apigw_client.create_api(
    Name=api_name,
    ProtocolType='HTTP',
    Target=lambda_arn
)
api_id = api_response['ApiId']
api_endpoint = api_response['ApiEndpoint']
print(f"API Gateway created with endpoint: {api_endpoint}")

# --- 7. Grant API Gateway permission to invoke Lambda ---
lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='apigateway-invoke',
    Action='lambda:InvokeFunction',
    Principal='apigateway.amazonaws.com',
    SourceArn=f"arn:aws:execute-api:{region}:{boto3.client('sts').get_caller_identity()['Account']}:{api_id}/*/*"
)
print("Granted API Gateway permission to invoke Lambda.")

print("\n🎉 Deployment Complete!")
print(f"Your API is live at: {api_endpoint}/")

Attached SageMaker policy to role: genmab-takehome-sagemaker-role
Waiting for IAM policy propagation...
Deleted existing Lambda function. Recreating...
Lambda function created with ARN: arn:aws:lambda:us-east-1:059006397895:function:sagemaker-proxy-lambda
API Gateway created with endpoint: https://30522nn89f.execute-api.us-east-1.amazonaws.com
Granted API Gateway permission to invoke Lambda.

🎉 Deployment Complete!
Your API is live at: https://30522nn89f.execute-api.us-east-1.amazonaws.com/


In [ ]:
# # --- Clean Up Script ---
# print("Starting cleanup...")
# try:
#     # Delete API Gateway
#     apis = apigw_client.get_apis()['Items']
#     for api in apis:
#         if api['Name'] == api_name:
#             apigw_client.delete_api(ApiId=api['ApiId'])
#             print(f"Deleted API Gateway: {api_name}")
#             break
            
#     # Delete Lambda Function
#     lambda_client.delete_function(FunctionName=lambda_function_name)
#     print(f"Deleted Lambda function: {lambda_function_name}")
    
#     # Detach policy from role
#     iam_client.detach_role_policy(RoleName=role_name, PolicyArn=sagemaker_invoke_policy_arn)
#     print(f"Detached SageMaker policy from role: {role_name}")
    
#     # Don't forget to delete your SageMaker endpoint if you haven't already!
#     predictor.delete_endpoint()

#     print("\n✅ Cleanup complete.")
# except Exception as e:
#     print(f"An error occurred during cleanup: {e}")